### Verifying and Validating a WOT acceleration model for the Porsche Taycan Turbo S ###
#### Presented by David Perner for the SAE Dynamical Modeling and Simulation Committee, April 21st 2020 ####

![2020-porsche-taycan.jpg](img/2020-porsche-taycan.jpg)

# Motivations #

* The Taycan is Porsche's first all-electric vehicle and is often set up as a competitor to the Tesla Model S
* Porsche has made an unusually large of technical data on the Taycan public, enabling some preliminary analysis
* The powertrain dynamics of electric vehicles are simpler than internal combustion engines, and much simpler than hybrids. 

In [1]:
using Unitful
using Interpolations
using DelimitedFiles
using Polynomials
using DifferentialEquations
using DataFrames

gn = ustrip(Unitful.gn);

A quick word on syntax. For those unfamiliar with it, the use of closures allows us to build what can be thought of as prototype functions, which when provided with more concrete values, return a function that can be called as intended. For instance:

In [2]:
proto_poly(a,b,c) = (x)->a*x^2 + b*x +c;

when provided with values of a, b, and c will return a function that evaluates the polynomial as a function of x. We could then take:

In [3]:
f = proto_poly(1,1,1)
(f(0), f(1), f(2)) 

(1, 3, 7)

This strategy allows for functions to be much more easily tested while retaining flexibility

Here are the equations for drag and rolling resistance using this strategy. The `@assert` macro only returns an error, so it should execute silently unless there's an issue.

In [4]:
drag(Cd, ρ, A) = (v)->0.5*Cd*A*ρ*v^2

drag_test=drag(2,1,1)
@assert drag_test(0) == 0
@assert drag_test(1) == 1
@assert drag_test(10) == 100
@assert drag_test(10.0) === 100.0

In [5]:
rolling_resistance(rr, mass) = (v)->v==0 ? 0 : rr*mass*gn

rr_test = rolling_resistance(1,1)

@assert rr_test(0)==0
@assert rr_test(1)==rr_test(2) "Rolling resistance is not constant"
@assert rr_test(-1)<0 "Rolling resistance is acting in the wrong direction when v<0"

AssertionError: AssertionError: Rolling resistance is acting in the wrong direction when v<0

According to the literature, the Taycan can only sustain its boosted output levels for 2.5 seconds. Therefore we need some "control" logic to enforce this constraint.

In [6]:
boost_limit(t_limit, boost_f, nominal_f) =
    (v,t) -> if t<t_limit
        boost_f(v)
    else
        nominal_f(v)
    end

traction_limit(f_capability, f_lim) = (v,t) -> min(f_capability(v,t), f_lim)

bl_test = boost_limit(1,(v)->1,(v)->20)
@assert bl_test(0,0)==1 
@assert bl_test(2,2)==20

trl_test = traction_limit((v,t)->v*t,10)
@assert trl_test(1,1)==1
@assert trl_test(1,20)==10

The total propulsive force on the vehicle is just the force exerted by the powertrain minus the drag and rolling resistance. Similarly, the acceleration is this total force divided by mass. While `@assert` testing wasn't added for these functions, they are very simply and testing could be added if necessary.

In [7]:
propulsive_force(mod_tq_f, aero_f, rr_f) =
    (v,t)-> mod_tq_f(v,t) - aero_f(v) - rr_f(v)

accel(m,pf) = (v,t) -> pf(v,t)/m;

The equation of state is a simple one, with the acceleration (`u[3]`) provided by the previous equations, and integrated once into velocity (`u[2]`) and again to displacement (`u[1]`). The test uses a constant acceleration of 1 and zero initial conditions, making the velocity $v = a*t$ and displacement $x=\frac{1}{2}a*t^2$.

In [8]:
eq_state(a) = (du, u, p, t) -> begin
    du[1] = u[2]
    du[2] = u[3]
    u[3] = a(u[2], t)
end

eqst_test = eq_state((x,y)->1)

u0 = [0.0, 0.0, 0.0]
tspan = (0.0, 10.0)
resp_test = solve(ODEProblem(eqst_test, u0, tspan))
@assert resp_test.u[end][3]==1
@assert isapprox(resp_test.u[end][2], 10, rtol=1e-5)
@assert isapprox(resp_test.u[end][1], 50, rtol=1e-5)

Now that we've tested the prototypical form of our physical and torque control equations, we can start providing the vehicle and test specific parameters

In [9]:
#Physical constants
ρ=1.225; #kg/m^3

#Vehicle parameters
mass_veh = 2_358.7; #kg
A_frontal = 2.331866; #m^2
Cd = 0.25;
tire_r_front = 0.3595; #in m, 21" rims, 265/35 tires
tire_r_rear = 0.358; # in m, 21" rims, 305/30 tires
boost_time = 2.5; #s, powertrain constraint

#Model calibration
ϕ_p = 0.99;
slip_ratio = 1.1;
rr = 0.01;

# Calculated constants
f_traction_max = ϕ_p * mass_veh * gn;

Porsche has also made this image publically available, showing the torque capabilities of the powertrain system. The data from the plot was pulled off using DataThief and scaled for Porsche's quoted boosted performance.

![taycan_pt_tq.png](img/taycan_pt_tq.png)

In [10]:
include("taycan_data_load.jl");

We can now create Taycan specific versions of our prototypical functions now that all the vehicle data has been specified and loaded in.

In [11]:
taycan_drag = drag(Cd, ρ, A_frontal)
taycan_rr = rolling_resistance(rr, mass_veh)
taycan_boost_mod_f = boost_limit(boost_time, boosted_pt_f, max_pt_f)
taycan_trc_lim_f = traction_limit(taycan_boost_mod_f, f_traction_max)
taycan_propulsive_f = propulsive_force(taycan_trc_lim_f, taycan_drag, taycan_rr)
taycan_accel = accel(mass_veh, taycan_propulsive_f)

taycan_state = eq_state(taycan_accel);

We can set up the integrator to solve the WOT equations of motion and can see that the resulting time is in the right ballpark and the end of the sim aligned with the target speed.

In [12]:
u0 = [0.0, 0.0, 0.0]
tspan=(0.0,30.0)

condition(u,t,integrator) = u[2] - 26.8224 #60 mph in m/s
affect!(integrator) = terminate!(integrator)
cb = ContinuousCallback(condition,affect!)
zeroTo60 = solve(
    ODEProblem(taycan_state, u0, tspan),
    Rodas4(autodiff=false), 
    saveat=1e-3, 
    reltol=1e-9, 
    callback=cb
)

(zeroTo60.t[end], zeroTo60.u[end][2])

(2.9326190138778334, 26.822399999999995)

To check that the model is consistent with physics, we need a measure distinct from the calculations performed so far. I have chosen to use energy conservation, and am comparing the kinetic energy at the end of the acceleration sim to the sum of its components of powertrain work and drag/rolling resistance losses using the numerically integrated distance and velocity histories.

In [13]:
veh_ke = 0.5*mass_veh*zeroTo60.u[end][2]^2

v = [zeroTo60.u[x][2] for x in 1:length(zeroTo60.u)]
d = [zeroTo60.u[x][1] for x in 1:length(zeroTo60.u)]
Δd = diff(d)

drag_work = sum(taycan_drag.(v[2:end]).*Δd)
rr_work = sum(taycan_rr.(v[2:end]).*Δd)
pt_work = sum(taycan_trc_lim_f.(v[2:end],zeroTo60.t[2:end]).*Δd)

energy_ϵ=(pt_work-drag_work-rr_work)/veh_ke-1

@assert abs(energy_ϵ)<1e-3

[Car and Driver](https://www.caranddriver.com/reviews/a29208876/2020-porsche-taycan-turbo-s-drive/) published numbers in late January with their testing results for the Taycan Turbo S. The data contain some quirks like the 0-60 time (2.4s) being less than 5-60 time (2.9s) and by a margin greater than the rollout time. Still, it provides some general sense of alignment and model accuracy.



In [ ]:
include("perf_comparison.jl")

In [14]:
display(perf_table)

,TestDescription,CarAndDriver,SimulationResults,AbsoluteError,RelativeError
,String,Float64,Float64,Float64,Float64
1,"Rollout, 1 ft (s)",0.2,0.251856,0.0518558,25.9279
2,60 mph (s),2.4,2.68076,0.280763,11.6985
3,100 mph (s),6.0,6.57714,0.577144,9.61907
4,150 mph (s),15.2,15.4345,0.234534,1.54298
5,"Rolling start, 5-60 mph (s)",2.9,2.69962,-0.200381,-6.90969
6,"Top gear, 30-50 mph (s)",1.1,0.935,-0.165,-15.0
7,"Top gear, 50-70 mph (s)",1.6,1.37,-0.23,-14.375
8,¼-mile time (s),10.5,10.8371,0.337144,3.2109
9,¼-mile speed (mph),130.0,128.02,-1.98003,-1.5231


## Further Validation Steps ##
Without more information about the test procedure or the test data itself, there's only so far that the model can be calibrated to published data without becoming overfit. And while some of the model predictions are not too far off, others are far outside the <5% error that may be acceptable for an early design model, with <1% error likely necessary for detailed, production level modeling. However, were such test data available, validating the model could include:
* Using vehicle velocity history as model input and compare predicted acceleration against
  * Traction limits and tire performance
  * Boost time limit torque reduction
  * Powertrain torque drop off due to tire slip
* Based on test conditions
  * Recompute equations of state based on more accurate initial conditions
  * Ensure simulation data is processed correctly
  * Account for post-processing adjustments

It is also possible that this model is too simple to properly capture the dynamics of WOT for this vehicle. Test data would also help point to model improvements to address these problems. Importantly, the model should only be made as complex as needed to capture the behavior to a required level of accuracy, and no more than that.  
